<a href="https://www.kaggle.com/code/dhruvpatel2411/hugging-face-ner?scriptVersionId=255537781" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Project Overview

This project demonstrates Named Entity Recognition (NER) using the CoNLL-2003 dataset. It leverages the Hugging Face datasets library for data loading and preprocessing, and uses a pre-trained DistilBERT model from the Transformers library fine-tuned for token classification.

### Workflow Summary

    Dataset: CoNLL-2003 (from Hugging Face datasets)

    Model: Pre-trained DistilBERT (distilbert-base-uncased) fine-tuned for token classification

    Preprocessing: Tokenization using DistilBertTokenizer with alignment of tokens to entity labels

    Fine-tuning: Hugging Face Trainer API with custom training arguments

    Evaluation: Model performance evaluated on the test set using token-level metrics such as F1 score, precision, and recall

    Classification Report: Detailed token classification metrics are generated during evaluation to analyze entity-level performance

### Inference

After fine-tuning, the model can be used to identify and classify named entities in new text samples. The inference process involves tokenizing input sentences, passing them through the fine-tuned model, and extracting predicted entity labels aligned with the input tokens, allowing the detection of entities such as persons, organizations, locations, and miscellaneous categories.

In [1]:
!pip install transformers
!pip install 'accelerate>=0.26.0'
!pip install -U datasets huggingface_hub
!pip install fsspec==2023.9.2

!pip install seqeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
from datasets import load_dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, f1_score
import numpy as np
import torch

2025-08-12 02:56:55.439207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754967415.790168      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754967415.894372      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Load dataset
dataset = load_dataset("wikiann", "en")

README.md: 0.00B [00:00, ?B/s]

en/validation-00000-of-00001.parquet:   0%|          | 0.00/748k [00:00<?, ?B/s]

en/test-00000-of-00001.parquet:   0%|          | 0.00/748k [00:00<?, ?B/s]

en/train-00000-of-00001.parquet:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [4]:
print(dataset)

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 10000
    })
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 20000
    })
})


In [5]:
label_list = dataset["train"].features["ner_tags"].feature.names
num_labels = len(label_list)

In [6]:
from transformers import DistilBertForTokenClassification
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-cased")
model = DistilBertForTokenClassification.from_pretrained("distilbert-base-cased", num_labels=num_labels)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Tokenize and align labels
# Label alignment function
def tokenize_and_align_labels(example):
    tokenized_inputs = tokenizer(
        example["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",  # or use padding=True for dynamic
        max_length=128,
    )

    labels = []
    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)
        elif word_idx != previous_word_idx:
            labels.append(example["ner_tags"][word_idx])
        else:
            labels.append(-100)  # Ignore subwords
        previous_word_idx = word_idx

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=False)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [8]:
from seqeval.metrics import classification_report, precision_score, recall_score, accuracy_score

# Compute metrics
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = []
    true_predictions = []

    for pred, label in zip(predictions, labels):
        current_labels = []
        current_preds = []

        for p_id, l_id in zip(pred, label):
            if l_id != -100:  # ignore special tokens
                current_labels.append(label_list[l_id])
                current_preds.append(label_list[p_id])

        true_labels.append(current_labels)
        true_predictions.append(current_preds)

    return {
        "accuracy": accuracy_score(true_labels, true_predictions),
    }

In [9]:
# Training
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    learning_rate=3e-5,
    weight_decay=0.01,
    report_to="none"
)

In [10]:
#Step 5: Fine-Tune the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

In [11]:
# Fine-tune the model
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.414100,0.277289,0.912874


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=625, training_loss=0.3899150817871094, metrics={'train_runtime': 176.6966, 'train_samples_per_second': 113.188, 'train_steps_per_second': 3.537, 'total_flos': 653324559360000.0, 'train_loss': 0.3899150817871094, 'epoch': 1.0})

In [12]:
#Step 6: Evaluate the Model
# Evaluate the fine-tuned model
trainer.evaluate()

{'eval_loss': 0.27728939056396484,
 'eval_accuracy': 0.9128743631166895,
 'eval_runtime': 26.1649,
 'eval_samples_per_second': 382.191,
 'eval_steps_per_second': 11.963,
 'epoch': 1.0}

### Inference

In [13]:
def predict_ner(text):
    import torch

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    tokens = text.split()

    # Tokenize with word alignment info
    encoded = tokenizer(tokens, return_tensors="pt", is_split_into_words=True, truncation=True)
    word_ids = encoded.word_ids(0)  # 🔥 get word_ids properly

    # Move inputs to device
    encoded = {k: v.to(device) for k, v in encoded.items()}

    with torch.no_grad():
        outputs = model(**encoded)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2)[0].cpu().numpy()

    final_predictions = []
    previous_word_idx = None

    for token_idx, word_idx in enumerate(word_ids):
        if word_idx is None or word_idx == previous_word_idx:
            continue  # Skip special tokens and subwords
        label_id = predictions[token_idx]
        label = label_list[label_id]
        final_predictions.append((tokens[word_idx], label))
        previous_word_idx = word_idx

    return final_predictions


In [14]:
# Test inference
print(predict_ner("Barack Obama was born in Hawaii and worked in Chicago."))

[('Barack', 'B-PER'), ('Obama', 'I-PER'), ('was', 'O'), ('born', 'O'), ('in', 'O'), ('Hawaii', 'B-LOC'), ('and', 'O'), ('worked', 'O'), ('in', 'O'), ('Chicago.', 'B-LOC')]


In [15]:
text = "Apple is looking at buying U.K. startup for $1 billion"
print(predict_ner(text))

[('Apple', 'B-ORG'), ('is', 'O'), ('looking', 'O'), ('at', 'O'), ('buying', 'O'), ('U.K.', 'B-ORG'), ('startup', 'O'), ('for', 'O'), ('$1', 'O'), ('billion', 'O')]


### Project Overview

This project demonstrates Named Entity Recognition (NER) using a token classification approach. It uses the Hugging Face datasets library for loading and preprocessing data (e.g., WikiAnn), and fine-tunes a pre-trained bert-base-cased model from the Hugging Face transformers library.

### Workflow Summary

    Dataset: WikiAnn (English) — from Hugging Face datasets hub

    Model: Pre-trained BERT (bert-base-cased) for token classification

    Preprocessing: Tokenization with alignment of entity tags using AutoTokenizer and is_split_into_words=True

    Fine-tuning: Hugging Face Trainer API with a custom training loop and token classification head

    Evaluation: Evaluation using seqeval metrics such as token-level accuracy and optionally precision, recall, and F1-score

    Confusion Matrix: Generated using predicted vs. true entity tags to visualize class-wise model performance

### Inference

Once trained, the model can be used to extract named entities from raw text. The inference pipeline:

    Tokenizes the input sentence using the same tokenizer as training

    Aligns token predictions back to original words

    Uses the model to generate predictions and maps the predicted label indices to human-readable NER tags (e.g., B-PER, I-LOC)

    Outputs a list of (word, entity) pairs